In [0]:
-- Top 10 peliculas mejores valoradas
SELECT
    m.title,
    g.genre,
    COUNT(f.rating) AS total_ratings,
    ROUND(AVG(f.rating), 2) AS avg_rating
FROM movielens.gold.fct_rating AS f
JOIN movielens.gold.dim_movie AS m ON f.movieId = m.movieId
LEFT JOIN movielens.gold.dim_genre AS g ON m.genre_id = g.genre_id
GROUP BY m.title, g.genre
HAVING COUNT(f.rating) >= 100
ORDER BY avg_rating DESC
LIMIT 10;

-- Ratings por genero
SELECT
    g.genre,
    COUNT(f.rating) AS total_ratings,
    ROUND(AVG(f.rating), 2) AS avg_rating
FROM movielens.gold.fct_rating AS f
JOIN movielens.gold.dim_movie AS m ON f.movieId = m.movieId
LEFT JOIN movielens.gold.dim_genre AS g ON m.genre_id = g.genre_id
GROUP BY g.genre
ORDER BY avg_rating DESC;

-- Top 10 usuarios más activos
SELECT
    u.userId,
    COUNT(f.rating) AS n_ratings,
    ROUND(AVG(f.rating), 2) AS avg_rating
FROM movielens.gold.fct_rating AS f
JOIN movielens.gold.dim_user AS u ON f.userId = u.userId
GROUP BY u.userId
ORDER BY n_ratings DESC
LIMIT 10;

-- Películas con mejor rating por género
WITH genre_rank AS (
    SELECT
        g.genre,
        m.title,
        COUNT(*) AS n_ratings,
        ROUND(AVG(f.rating), 2) AS avg_rating,
        ROW_NUMBER() OVER (PARTITION BY g.genre ORDER BY AVG(f.rating) DESC, COUNT(*) DESC) AS rn
    FROM movielens.gold.fct_rating AS f
    JOIN movielens.gold.dim_movie AS m ON f.movieId = m.movieId
    LEFT JOIN movielens.gold.dim_genre AS g ON m.genre_id = g.genre_id
    GROUP BY g.genre, m.title
)
SELECT genre, title, n_ratings, avg_rating
FROM genre_rank
WHERE rn <= 3
ORDER BY genre, avg_rating DESC;